In [1]:
import pandas as pd
import gensim

C:\Users\keiko\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import nltk
from nltk.corpus import stopwords

In [3]:
import string
from string import punctuation

In [4]:
beige = pd.read_csv('beige_books.csv')

In [ ]:
#for removing stop words/stemming words
%%capture
!pip install spacy
!python -m spacy download en
!pip install --no-cache-dir wordcloud
!pip3 install --no-cache-dir -U folium
!pip install --no-cache-dir textblob
from datascience import make_array
from datascience import *
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
%matplotlib inline
import folium
import pandas as pd
from IPython.display import HTML, display, IFrame
import folium
import spacy
from wordcloud import WordCloud

## Preprocessing

In [5]:
beige.head()

,ID,Date,District,Sector Heading,Sector Text
0,1,5-20-1970,Atlanta,Summary of Economic Activity,The mood of our directors varies from pessimis...
1,2,5-20-1970,Boston,Summary of Economic Activity,Discussions Monday (May 18) and last Friday wi...
2,3,5-20-1970,Chicago,Summary of Economic Activity,In published statements and private conversati...
3,4,5-20-1970,Cleveland,Summary of Economic Activity,Economic activity in the District has been dis...
4,5,5-20-1970,Dallas,Summary of Economic Activity,Continued inflation is the major concern of El...


In [6]:
print(len(beige['Sector Text']))
texts_series = beige['Sector Text']
texts = list(beige['Sector Text'])

25273


#### Getting rid of null vals, punctuation marks, stop words, and lowercasing

In [29]:
print(len(beige[texts_series.isnull()]))
beige[texts_series.isnull()]

139


,ID,Date,District,Sector Heading,Sector Text
20,21,5-20-1970,San Francisco,Summary for Selected Economic Aspects,NaN
29,30,5-20-1970,San Francisco,Inventories,NaN
30,31,5-20-1970,San Francisco,Both higher and lower inventories are reported.,NaN
92,93,10-14-1970,Atlanta,five-week delivery.,NaN
121,122,12-9-1970,Cleveland,one- and two-year maturity ranges.,NaN
135,136,1-12-1971,Dallas,as a whole.,NaN
216,217,7-21-1971,San Francisco,August 1.,NaN
258,259,11-10-1971,Boston,Regulation Q.,NaN
584,585,2-13-1974,Richmond,will decline.,NaN
628,629,6-12-1974,Dallas,just 6 percent.,NaN


In [7]:
#get rid of empty text records
beige.dropna(axis = 0, how = 'any', inplace = True)
len(beige)

25134

In [31]:
type(beige['Sector Text'][0])

str

In [8]:
#lowercase all records
beige['Sector Text'] = beige['Sector Text'].str.lower()

In [9]:
beige['Sector Text'][0]

"the mood of our directors varies from pessimism to optimism. if any consensus exists, it is that business activity in the sixth district will continue to weaken slightly. many major economic indices should drift downward or mark time for the next two months at least.in the pessimistic vein, a leading department store recently informed us that their sales are even with a year ago, but only because of the addition of branches; unit sales, of course, are lower. the store reported that labor costs were up 8 to 10 percent and profits were off. they were pessimistic about future department store sales and did not anticipate a quick economic recovery. a telephone survey of department stores indicates this conclusion is shared by many retailers throughout the southeast.according to directors' reports, retail sales in some areas have been adversely affected because of a diversion of purchasing power to private schools which have sprung up in response to integration orders. lumber mills in many

#### getting rid of punctuation marks

In [10]:
import re

In [35]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
ex = 'this @(*&is an exmaple text *&^%@^}{(0} to use'
re.sub('[^\s\w]', '', ex)

'this is an exmaple text 0 to use'

In [11]:
beige['Sector Text'].str.replace(r'[^\w\s]', '')
beige['Sector Text'] = beige['Sector Text'].str.replace(r'[^\w\s]', '')

In [ ]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [12]:
beige['District'].unique()

array(['Atlanta', 'Boston', 'Chicago', 'Cleveland', 'Dallas',
       'Kansas City', 'Minneapolis', 'New York', 'Philadelphia',
       'Richmond', 'San Francisco', 'St Louis', 'District'], dtype=object)

In [46]:
#ny = beige[beige['District'] == 'New York']
#ny.to_csv(path_or_buf = 'ny.csv', sep = ',', encoding='utf-8')

#### getting rid of stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\keiko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
stop_words = stopwords.words('english')

In [16]:
#remove stop words/tokenize
def rem_stop(text):
    
    from nltk.corpus import stopwords
    
    stop_words = stopwords.words("english")
       
    toks = nltk.word_tokenize(text)
    toks_reduced = [x for x in toks if x.lower() not in stopwords.words('english')]
    
    return toks_reduced 

In [56]:
len(beige['Sector Text'])

25134

In [17]:
beige['tokenized'] = [rem_stop(t) for t in beige['Sector Text']]

In [18]:
beige.to_csv(path_or_buf = 'beige_tokenized.csv', sep = ',', encoding='utf-8')

In [6]:
token = pd.read_csv('beige_tokenized.csv')
token.head()



,Unnamed: 0,ID,Date,District,Sector Heading,Sector Text,tokenized
0,0,1,5-20-1970,Atlanta,Summary of Economic Activity,the mood of our directors varies from pessimis...,"['mood', 'directors', 'varies', 'pessimism', '..."
1,1,2,5-20-1970,Boston,Summary of Economic Activity,discussions monday may 18 and last friday with...,"['discussions', 'monday', 'may', '18', 'last',..."
2,2,3,5-20-1970,Chicago,Summary of Economic Activity,in published statements and private conversati...,"['published', 'statements', 'private', 'conver..."
3,3,4,5-20-1970,Cleveland,Summary of Economic Activity,economic activity in the district has been dis...,"['economic', 'activity', 'district', 'disrupte..."
4,4,5,5-20-1970,Dallas,Summary of Economic Activity,continued inflation is the major concern of el...,"['continued', 'inflation', 'major', 'concern',..."


In [ ]:
len(token['tokenized'][0])

#### using spaCy to handle same-stem words

In [39]:
#!python -m spacy download en

In [19]:
#download en on spaCy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [44]:
parsed = nlp(beige['Sector Text'][0].lower())

In [45]:
def tablefy(parsed_text):
    toks_tab = pd.DataFrame()
    toks_tab['Word'] = [word.text for word in parsed_text]
    toks_tab['POS'] = [word.pos_ for word in parsed_text]
    toks_tab['Lemma'] = [word.lemma_ for word in parsed_text]
    toks_tab['Stop Word']=[word.is_stop for word in parsed_text]
    toks_tab['Punctuation']=[word.is_punct for word in parsed_text]
    toks_tab['Space']=[word.is_space for word in parsed_text]
    toks_tab['Number']=[word.like_num for word in parsed_text]
    toks_tab['OOV']=[word.is_oov for word in parsed_text]
    toks_tab['Dependency']=[word.dep_ for word in parsed_text]
    return toks_tab

In [73]:
def tablefyjr(parsed_text):
    toks_tab = pd.DataFrame()
    toks_tab['Word'] = [word.text for word in parsed_text]
    toks_tab['Lemma'] = [word.lemma_ for word in parsed_text]
    toks_tab['Stop Word']=[word.is_stop for word in parsed_text]
    return toks_tab

In [46]:
tablefy(parsed).head()

,Word,POS,Lemma,Stop Word,Punctuation,Space,Number,OOV,Dependency
0,the,DET,the,True,False,False,False,True,det
1,mood,NOUN,mood,False,False,False,False,True,nsubj
2,of,ADP,of,True,False,False,False,True,prep
3,our,ADJ,-PRON-,True,False,False,False,True,poss
4,directors,NOUN,director,False,False,False,False,True,pobj


In [32]:
[word.text for word in parsed]

['The',
 'mood',
 'of',
 'our',
 'directors',
 'varies',
 'from',
 'pessimism',
 'to',
 'optimism',
 '.',
 'If',
 'any',
 'consensus',
 'exists',
 ',',
 'it',
 'is',
 'that',
 'business',
 'activity',
 'in',
 'the',
 'Sixth',
 'District',
 'will',
 'continue',
 'to',
 'weaken',
 'slightly',
 '.',
 'Many',
 'major',
 'economic',
 'indices',
 'should',
 'drift',
 'downward',
 'or',
 'mark',
 'time',
 'for',
 'the',
 'next',
 'two',
 'months',
 'at',
 'least',
 '.',
 'In',
 'the',
 'pessimistic',
 'vein',
 ',',
 'a',
 'leading',
 'department',
 'store',
 'recently',
 'informed',
 'us',
 'that',
 'their',
 'sales',
 'are',
 'even',
 'with',
 'a',
 'year',
 'ago',
 ',',
 'but',
 'only',
 'because',
 'of',
 'the',
 'addition',
 'of',
 'branches',
 ';',
 'unit',
 'sales',
 ',',
 'of',
 'course',
 ',',
 'are',
 'lower',
 '.',
 'The',
 'store',
 'reported',
 'that',
 'labor',
 'costs',
 'were',
 'up',
 '8',
 'to',
 '10',
 'percent',
 'and',
 'profits',
 'were',
 'off',
 '.',
 'They',
 'were',
 

# Using Clean Beige Books Data!

In [49]:
clean = pd.read_csv('cleaned.csv')

In [50]:
clean.head()

,ID,Date,District,Sector Heading,Sector Text
0,1,5-20-1970,Atlanta,Summary of Economic Activity,The mood of our directors varies from pessimis...
1,2,5-20-1970,Boston,Summary of Economic Activity,Discussions Monday (May 18) and last Friday wi...
2,3,5-20-1970,Chicago,Summary of Economic Activity,In published statements and private conversati...
3,4,5-20-1970,Cleveland,Summary of Economic Activity,Economic activity in the District has been dis...
4,5,5-20-1970,Dallas,Summary of Economic Activity,Continued inflation is the major concern of El...


In [51]:
#lowercase
clean['text'] = clean['Sector Text'].str.lower()

In [53]:
#punctuation
clean['text'] = clean['text'].str.replace(r'[^\w\s]', '')

In [54]:
clean.head()

,ID,Date,District,Sector Heading,Sector Text,text
0,1,5-20-1970,Atlanta,Summary of Economic Activity,The mood of our directors varies from pessimis...,the mood of our directors varies from pessimis...
1,2,5-20-1970,Boston,Summary of Economic Activity,Discussions Monday (May 18) and last Friday wi...,discussions monday may 18 and last friday with...
2,3,5-20-1970,Chicago,Summary of Economic Activity,In published statements and private conversati...,in published statements and private conversati...
3,4,5-20-1970,Cleveland,Summary of Economic Activity,Economic activity in the District has been dis...,economic activity in the district has been dis...
4,5,5-20-1970,Dallas,Summary of Economic Activity,Continued inflation is the major concern of El...,continued inflation is the major concern of el...


In [69]:
words[words['Word'] == 'area']

,Word,POS,Lemma,Stop Word,Punctuation,Space,Number,OOV,Dependency
245,area,NOUN,area,False,False,False,False,True,pobj
467,area,NOUN,area,False,False,False,False,True,pobj
480,area,NOUN,area,False,False,False,False,True,compound
499,area,NOUN,area,False,False,False,False,True,pobj
541,area,NOUN,area,False,False,False,False,True,attr


In [72]:
parsed = nlp(clean['text'][0])
words = tablefy(parsed)
count= words[words['Stop Word'] == False].groupby('Lemma').count()
count.head()

,Word,POS,Stop Word,Punctuation,Space,Number,OOV,Dependency
Lemma,,,,,,,,
10,1,1,1,1,1,1,1,1
2600,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1
act,1,1,1,1,1,1,1,1
activity,2,2,2,2,2,2,2,2


In [ ]:
#get rid of stop words
def rem_stop(text):
    
    from nltk.corpus import stopwords
    
    stop_words = stopwords.words("english")
       
    toks = nltk.word_tokenize(text)
    toks_reduced = [x for x in toks if x.lower() not in stopwords.words('english')]
    
    return toks_reduced 

In [91]:
#make single list of all clean words in cleaned.csv
nostop_stem = []
for t in clean['text']:
    parsed = nlp(t)
    tab = tablefyjr(parsed)
    nostop_stem += list(tab[tab['Stop Word'] == False]['Lemma'])

ValueError: Length of values does not match length of index

In [107]:
table = pd.DataFrame()
table['string'] = nostop_stem
table.head()

,string
0,mood
1,director
2,vary
3,pessimism
4,optimism


In [109]:
table.to_csv(path_or_buf = 'noSS_onelist.csv', sep = ',', encoding='utf-8')

In [101]:
import numpy as np

In [115]:
#make list of lists, each list element relating to one record
nostop_stem2 = []
for t in clean['text']:
    parsed = nlp(t)
    tab = tablefyjr(parsed)
    stems = [tab[tab['Stop Word'] == False]['Lemma']]
    nostop_stem2.append(stems)

In [116]:
clean['nostopstem'] = nostop_stem2

In [ ]:
#clean.to_csv(path_or_buf = 'cleaned_nostopstem.csv', sep = ',', encoding='utf-8').

In [123]:
list(clean['nostopstem'][3][0])

['economic',
 'activity',
 'district',
 'disrupt',
 'past',
 'week',
 'largely',
 'trucking',
 'strike',
 'strike',
 'major',
 'rubber',
 'company',
 'widespread',
 'campus',
 'disturbance',
 'district',
 'force',
 'number',
 'university',
 'close',
 'undoubtedly',
 'adverse',
 'impact',
 'business',
 'condition',
 'local',
 'area',
 'bank',
 'regular',
 'monthly',
 'survey',
 '70',
 'large',
 'manufacturing',
 'company',
 'district',
 'reflect',
 'softening',
 'nationwide',
 'industrial',
 'activity',
 'fall',
 'early',
 'winter',
 'sign',
 'bottoming',
 'evidence',
 'survey',
 'take',
 'april',
 'survey',
 'reveal',
 'sharp',
 'setback',
 'month',
 'april',
 'key',
 'series',
 'new',
 'order',
 'shipment',
 'marked',
 'deterioration',
 'employment',
 'workweek',
 'presumably',
 'work',
 'stoppag',
 'district',
 'insure',
 'unemployment',
 'rate',
 'edge',
 'moderately',
 'upward',
 '1',
 'percent',
 'october',
 '2',
 'percent',
 'late',
 'march',
 'jump',
 'percentage',
 'point',
 'a

In [127]:
len([i for i in np.arange(len(clean))])

25122

In [128]:
clean['nostop_stem'] = [list(clean['nostopstem'][i][0]) for i in np.arange(len(clean))]

In [137]:
clean.drop('Sector Text', axis=1, inplace=True)

In [138]:
clean.head()

,ID,Date,District,Sector Heading,nostop_stem
0,1,5-20-1970,Atlanta,Summary of Economic Activity,"[mood, director, vary, pessimism, optimism, co..."
1,2,5-20-1970,Boston,Summary of Economic Activity,"[discussion, monday, 18, friday, district, res..."
2,3,5-20-1970,Chicago,Summary of Economic Activity,"[publish, statement, private, conversation, de..."
3,4,5-20-1970,Cleveland,Summary of Economic Activity,"[economic, activity, district, disrupt, past, ..."
4,5,5-20-1970,Dallas,Summary of Economic Activity,"[continue, inflation, major, concern, eleventh..."


In [139]:
clean.to_csv(path_or_buf = 'noSS_table.csv', sep = ',', encoding='utf-8')

In [142]:
pd.read_csv('beige_tokenized.csv')

,Unnamed: 0,ID,Date,District,Sector Heading,Sector Text,tokenized
0,0,1,5-20-1970,Atlanta,Summary of Economic Activity,the mood of our directors varies from pessimis...,"['mood', 'directors', 'varies', 'pessimism', '..."
1,1,2,5-20-1970,Boston,Summary of Economic Activity,discussions monday may 18 and last friday with...,"['discussions', 'monday', 'may', '18', 'last',..."
2,2,3,5-20-1970,Chicago,Summary of Economic Activity,in published statements and private conversati...,"['published', 'statements', 'private', 'conver..."
3,3,4,5-20-1970,Cleveland,Summary of Economic Activity,economic activity in the district has been dis...,"['economic', 'activity', 'district', 'disrupte..."
4,4,5,5-20-1970,Dallas,Summary of Economic Activity,continued inflation is the major concern of el...,"['continued', 'inflation', 'major', 'concern',..."
5,5,6,5-20-1970,Kansas City,Summary of Economic Activity,consensus based on discussions with members of...,"['consensus', 'based', 'discussions', 'members..."
6,6,7,5-20-1970,Minneapolis,Summary of Economic Activity,although indications of softening in the ninth...,"['although', 'indications', 'softening', 'nint..."
7,7,8,5-20-1970,New York,Summary of Economic Activity,in the second federal reserve district the boa...,"['second', 'federal', 'reserve', 'district', '..."
8,8,9,5-20-1970,Philadelphia,Summary of Economic Activity,the business slowdown in the regional economy ...,"['business', 'slowdown', 'regional', 'economy'..."
9,9,10,5-20-1970,Richmond,"Production, Sales, and Inventories",district survey respondents except for those i...,"['district', 'survey', 'respondents', 'except'..."


# Topic Modelling

In [1]:
beige = pd.read_csv('noSS_table')

NameError: name 'pd' is not defined

In [ ]:
beige.head()